In [1]:
import scipy.io
import numpy as np
import scipy as sp
import cupy as cp
import matplotlib.pyplot as plt
import os
from SWE_utils_cupy import *
from scipy.signal import firwin, butter, buttord, freqz

from arrus.ops.us4r import *
from arrus.ops.imaging import *
from arrus.metadata import *
from arrus.devices.probe import *
from arrus.devices.us4r import Us4RDTO
from arrus.utils.imaging import *

In [2]:
print(f"Device count: {cp.cuda.runtime.getDeviceCount()}")
device_props = cp.cuda.runtime.getDeviceProperties(0)

print(f"Device: {device_props['name']} (cc {device_props['major']}.{device_props['minor']})")
print(f"GPU clock frequency: {device_props['clockRate']/1e3} MHz")
print(f"SM Count: {device_props['multiProcessorCount']}")
print("Available memory: ")
print(f"- global memory: {device_props['totalGlobalMem']/2**20} MiB")
print(f"- shared memory per thread block: {device_props['sharedMemPerBlock']} B")
print(f"- constant memory: {device_props['totalConstMem']} B")

Device count: 1
Device: b'GeForce RTX 3060 Laptop GPU' (cc 8.6)
GPU clock frequency: 1425.0 MHz
SM Count: 30
Available memory: 
- global memory: 5946.5625 MiB
- shared memory per thread block: 49152 B
- constant memory: 65536 B


# Script parameters
Configure the reconstruction parameters of individual steps of the algorithm

In [3]:
# General settings
dataset_id = 340
directory  = '../Datasets'

# Constants
c              = 1540.0
probe_elements = 128
probe_pitch    = 0.298e-3
fs             = 65e6

# Sequence parameters
pwi_txFreq   = 4.4e6
pwi_nCycles  = 2
pwi_txAngles = [-4.0, 0.0, 4.0]
pwi_txPri    = 100e-6
pwi_fri      = 2* pwi_txPri

# RF Filter
rf_filter_band        = [4e6, 7e6]    # Desired pass band, Hz
rf_filter_trans_width = 1e6           # Width of transition from pass band to stop band, Hz
rf_filter_numtaps     = 236           # Size of the FIR filter.

# Post down conversion IQ filtering
demod_filter_cutoff = 0.5 * 4.4e6       # Desired cutoff frequency, Hz
demod_filter_trans_width = 0.5 * 4.4e6  # Width of transition from pass band to stop band, Hz
demod_filter_numtaps = 128              # Size of the FIR filter.

# Beamforming
px_size = 0.2
x_grid = np.arange(-20, 20, px_size) * 1e-3
z_grid = np.arange(0, 50, px_size)   * 1e-3

# Shear wave detection
swd_mode              = 'kasai'
swd_zGate_length      = 4 
swd_ensemble_length   = 4

# Input parameters
df_sws_range = [0.5, 4.0];
df_f_range   = [40.0, 1000.0];
df_k_range   = 0.9;

# SWS estimation
swse_interp_factor = 10;
swse_interp_order  = 2
swse_d             = 14;
swse_frames        = [0, 99];
swse_SWV_range     = [0.5, 4.0];
swse_x_range       = [[0, 420], [0, 420]]

# Post-processing
median_filter_size = 10



## Load the dataset

In [4]:
# Load a dataset
sid = 'id' + str(dataset_id)
# Search for the file
for root, dirs, files in os.walk(directory):
    for file in files:
        if sid in file:
            file_path = os.path.join(root, file)
            data = sp.io.loadmat(file_path)

data["rf_data"].shape
rf = data["rf_data"]
rf = rf[np.newaxis, ...]
Nframes = rf.shape[1]
rf.shape

(1, 150, 4224, 128)

# Beamforming

### Design filters

In [5]:
## Function to plot the filter response
def plot_FIR_response(fs, taps, w, h):
    "Utility function to plot response functions"
    fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(8, 9))
    ax0.plot(range(len(taps)), taps)
    ax0.set_xlabel("Tap")
    ax0.set_ylabel("Value")
    
    ax1.plot(0.5*fs*w/np.pi, 20*np.log10(np.abs(h)))
    ax1.set_ylim(-80, 5)
    ax1.set_xlim(0, 0.5*fs)
    ax1.grid(True)
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('Gain (dB)')
    
    ax2.plot(0.5*fs*w/np.pi, np.angle(h))
    ax2.set_xlim(0, 0.5*fs)
    ax2.set_xlabel("Frequency (Hz)")
    ax2.set_ylabel("Phase (rad)")

## Design a band-pass FIR filter for RF filtering of raw channel data
band = rf_filter_band                # Desired pass band, Hz
trans_width = rf_filter_trans_width  # Width of transition from pass band to stop band, Hz
numtaps = rf_filter_numtaps          # Size of the FIR filter.

edges = [0, band[0] - trans_width, band[0], band[1], band[1] + trans_width, 0.5*fs]
rf_fir_taps = signal.remez(numtaps, edges, [0, 1, 0], Hz=fs)
if(0):
    w, h = signal.freqz(rf_fir_taps, [1], worN=2000)
    plot_FIR_response(fs, rf_fir_taps, w, h)

## Design a low-pass FIR filter for filtering of down-conversion products
# Specify the filter parameters    
cutoff = demod_filter_cutoff            # Desired cutoff frequency, Hz
trans_width = demod_filter_trans_width  # Width of transition from pass band to stop band, Hz
numtaps = demod_filter_numtaps          # Size of the FIR filter.
iq_fir_taps = signal.remez(numtaps, [0, cutoff, cutoff + trans_width, 0.5*fs], [1, 0], Hz=fs)
if(0):
    w, h = signal.freqz(iq_fir_taps, [1], worN=2000)
    plot_FIR_response(fs, iq_fir_taps, w, h)           

## Metadata

In [6]:
tx_angles = np.tile(pwi_txAngles, int(np.ceil(rf.shape[1]/len(pwi_txAngles))))*np.pi/180

sequence = PwiSequence(
    pulse=Pulse(center_frequency=pwi_txFreq, n_periods=pwi_nCycles, inverse=False),
    rx_sample_range=(0, rf.shape[1]),
    speed_of_sound=c, # [m/s],
    angles=tx_angles,
    pri=pwi_txPri
)

model=ProbeModel(
    model_id=ProbeModelId("atl", "l7-4"),
    n_elements=probe_elements,
    pitch=probe_pitch,
    curvature_radius=0,
)

probe = ProbeDTO(
    model=model
)

device=Us4RDTO(
    sampling_frequency=fs,
    probe=probe
)

context = arrus.metadata.FrameAcquisitionContext(
    device=device, 
    sequence=sequence,
    raw_sequence=None,
    medium=None,
    custom_data={})

data_desc=EchoDataDescription(
    sampling_frequency=fs,
)

metadata = ConstMetadata(context=context, data_desc=data_desc, input_shape=rf.shape, is_iq_data=False, dtype=np.int16, version=None)

In [7]:
! nvidia-smi

Mon Jan 23 13:09:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8    20W /  N/A |    281MiB /  5946MiB |     24%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Pipeline: RF Filtering, Down-conversion and Beamforming

In [8]:
# Define the processing pipeline
processing = Pipeline(
    steps=(
        Transpose(axes=(0, 1, 3, 2)),
        #BandpassFilter(),
        FirFilter(taps=rf_fir_taps, num_pkg=None, filter_pkg=None),
        QuadratureDemodulation(),
        Decimation(filter_type="fir", filter_coeffs=iq_fir_taps, decimation_factor=1),
        ReconstructLri(x_grid=x_grid, z_grid=z_grid),
        Squeeze(),
        # SWE
        #Pipeline(
        #    steps=(
        #        SWDetection(),
                # ...
        #    )
        #),
        # B-mode
        #Mean(axis=0),
        #EnvelopeDetection(),
        #LogCompression(),
        # SWDetection()
    ),
    placement="/GPU:0"
)

# Prepare pipeline
processing.prepare(metadata)


deque([<arrus.metadata.ConstMetadata at 0x7f7ff500b100>])

In [9]:
#%%timeit -r 10 -n 10
# Run the pipeline
output = processing.process(cp.asarray(rf))

In [10]:
lri_data_gpu = output[0]
print(lri_data_gpu.shape)

(150, 200, 250)


In [11]:
! nvidia-smi

Mon Jan 23 13:09:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8    27W /  N/A |   2338MiB /  5946MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Angle compounding

In [12]:
# Compounding
AngleCompounder = AngleCompounding(nAngles=len(pwi_txAngles), axis=0)
AngleCompounder.prepare()

In [13]:
%%time
2+2

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.81 µs


4

In [18]:
%%time
import time

n = 100
start = time.time()
for i in range(n):
    hri_data_gpu = AngleCompounder.process(data=lri_data_gpu)
cp.cuda.Stream.null.synchronize()
end = time.time()
print(f"Average time: {(end-start)/n} [s]")

Average time: 0.01899691343307495 [s]
CPU times: user 1.9 s, sys: 0 ns, total: 1.9 s
Wall time: 1.9 s


In [20]:
%%timeit -r 10 -n 100
hri_data_gpu = AngleCompounder.process(data=lri_data_gpu)

The slowest run took 167.70 times longer than the fastest. This could mean that an intermediate result is being cached.
10.2 ms ± 6.78 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [ ]:
! nvidia-smi

### B-mode display

In [ ]:
BmodeGenerator = GenerateBmode()
BmodeGenerator.prepare()
Bmode = BmodeGenerator.process(data=hri_data_gpu[5, :, :]) # Done on GPU
BmodeGenerator.plotHistogram(Bmode)
BmodeGenerator.displayBmode(frame=Bmode, dB_range=[40, 80])


# Shear wave detection

In [ ]:
! nvidia-smi

In [ ]:
del lri_data_gpu
del processing
cp._default_memory_pool.free_all_blocks()

In [ ]:
# Crop the data
hri_data_gpu = cp.moveaxis(hri_data_gpu, [0,1], [2,1])

In [ ]:
# Shear wave detection
swd_mode              = 'kasai'
swd_zGate_length      = 8 
swd_ensemble_length   = 8

In [ ]:
# Shear wave detection
ShearDetector = ShearwaveDetection(mode=swd_mode, packet_size=swd_ensemble_length, z_gate=swd_zGate_length, frame_pri=pwi_fri, c=c, fc=pwi_txFreq, fs=fs)
ShearDetector.prepare()

In [ ]:
print(hri_data_gpu.shape)

In [ ]:
#%%timeit -r 10 -n 100
ddata = ShearDetector.process(data=hri_data_gpu)

In [ ]:
print(ddata.shape)

In [ ]:
del hri_data_gpu
cp._default_memory_pool.free_all_blocks()

In [ ]:
ddata_cpu = ddata.get()
getHistogram(data=ddata_cpu, Vrange = [-2.5e-3, 2.5e-3])
print(ddata_cpu.shape)
DrawShearWaveFrames(data=ddata_cpu, Vrange=[-2e-3, 4e-3], frames=[1, 4, 7, 10, 13, 16])

In [ ]:
# Plot some profiles
#%matplotlib widget
#fig, ax = plt.subplots(1, 1, figsize=(8, 4))
#ax.plot(np.squeeze(ddata_cpu[300, 320-15, :]))
#ax.plot(np.squeeze(ddata_cpu[300, 320+15, :]))

# Shear wave motion data filtering

In [ ]:
print(ddata.shape)

In [ ]:
# Shear wave motion data filtering in Fourier domain
DirFilter = ShearwaveMotionDataFiltering(data_shape = ddata.shape, sws_range=df_sws_range, f_range=df_f_range, k_range=df_k_range, fs=1.0/pwi_fri)
DirFilter.prepare()

In [ ]:
#%%timeit -r 10 -n 100
(ddata_LR, ddata_RL) = DirFilter.process(data=ddata)

In [ ]:
print(ddata_LR.shape)

In [ ]:
! nvidia-smi

In [ ]:
# Free GPU memory
del DirFilter
del ShearDetector
del ddata
cp._default_memory_pool.free_all_blocks()

In [ ]:
## Visualize results
# Filter masks
#DirFilter.plotFilterMasks()
# Filtered motion data frames
DrawShearWaveFrames(data=ddata_cpu,    Vrange=[-2e-3, 4e-3], frames=[1, 4, 7, 10, 13, 16])
DrawShearWaveFrames(data=ddata_LR.get(), Vrange=[-2e-3, 4e-3], frames=[1, 4, 7, 10, 13, 16])
DrawShearWaveFrames(data=ddata_RL.get(), Vrange=[-2e-3, 4e-3], frames=[1, 4, 7, 10, 13, 16])
# Plot some profiles
#%matplotlib widget
#fig, ax = plt.subplots(1, 1, figsize=(8, 4))
#ax.plot(np.squeeze(ddata_LR[200, 350-15, :]))
#ax.plot(np.squeeze(ddata_LR[200, 350+15, :]))

# Local shear wave speed estimation

## FFT-based correlation approach

In [ ]:
class SWS_Estimation():
    # Expected data format: 2x 3D array [2, z, x, frame]
    def __init__(self, data_shape, x_range, z_clip, frames_range, d, interp_factor, interp_order=3, px_pitch=0.1e-3, FRI =200e-6, sws_range=[0.5, 4.0]):
        # Params capture
        self.out_dim = [data_shape[0], data_shape[1], data_shape[2]] # of one sub-dataset
        self.x_range = x_range # for example [[0, 200], [250, 400]], so x_ranges for RL and LR datasets: [[xL_LR, xR_LR], [xL_RL, xR_RL]]
        self.frames_range = frames_range # for example [0, 99]
        self.z_clip = z_clip # for example [10, 30] - how many pixels to clip from top and bottom
        self.d = d
        self.interp_factor = interp_factor
        self.interp_order = interp_order
        self.px_pitch = px_pitch
        self.FRI = FRI
        self.sws_range = sws_range
    
    def prepare(self):
        
        # Process column ranges 
        self.xL = [0, 0]
        self.xR = [0, 0]       
        
        # LR
        self.xL[0] = self.x_range[0][0] - int(np.ceil(self.d/2)) 
        self.xR[0] = self.x_range[0][1] + int(np.ceil(self.d/2)) 
        if(self.xL[0] < 0):
            self.xL[0] = 0;

        if(self.xR[0] > self.out_dim[1]):
            self.xR[0] = self.out_dim[1] 
            
        # RL
        self.xL[1] = self.x_range[1][0] - int(np.ceil(self.d/2)) 
        self.xR[1] = self.x_range[1][1] + int(np.ceil(self.d/2)) 
        if(self.xL[1] < 0):
            self.xL[1] = 0;

        if(self.xR[1] > self.out_dim[1]):
            self.xR[1] = self.out_dim[1]             
            
        # Get window function
        #N = self.out_dim[2] * self.interp_factor
        N = (self.frames_range[1] - self.frames_range[0]) * self.interp_factor
        w = sp.signal.windows.tukey(N, alpha=0.2, sym=True)
        self.w = cp.asarray(w)
        
        # Other
        self.ds = self.px_pitch * self.d
        
        return  
    
    def process(self, data):
        
        # Create output buffer
        SWV = cp.zeros((2, 2, self.out_dim[0], self.out_dim[1]))  #[SWV/r, LR/RL,  z, x]
        
        # Process each sub-dataset (LR, and RL independently)
        for i in range(2):
            # Unpack data
            ddata = cp.squeeze(data[i])
        
            # Crop data
            ddata = ddata[self.z_clip[0]:-self.z_clip[1], self.xL[i]:self.xR[i], self.frames_range[0]:self.frames_range[1]]
            ddata[cp.isnan(ddata)]=0
        
            # Data interpolation along slow-time dimension
            print(ddata.shape)
            ynew = cx.ndimage.zoom(input=ddata, zoom=(1, 1, self.interp_factor), output=data.dtype, order=self.interp_order)
        
            # DC offset cancellation
            m = cp.mean(ynew, axis=2)
            ynew = cp.transpose(ynew, [2,0,1])
            ynew = cp.subtract(ynew, m)
            ynew = cp.transpose(ynew, [1,2,0])
        
            # Apply window function
            ynew = ynew * self.w

            # Find normalization values
            txa = ynew[:, 0:ynew.shape[1]-self.d, :]
            txb = ynew[:, self.d:ynew.shape[1], :]
            Rxa = cp.sum(txa * txa, axis=2)
            Rxb = cp.sum(txb * txb, axis=2)

            # Compute the FFT along the slow-time dim
            X = cx.fft.rfft(ynew, axis=2, overwrite_x=True)

            # Correlation in frequency domain and back to time-domain
            Xa = X[:, 0:X.shape[1]-self.d, :]
            Xb = X[:, self.d:X.shape[1], :]
            c = cp.real(cx.fft.irfft(Xa * cp.conj(Xb), axis=2, overwrite_x=True))

            # Shift zero
            c = cp.concatenate((c[:, :, c.shape[2]//2:], c[:, :, :c.shape[2]//2]), axis=2)

            # Normalize the correlation values
            c = cp.transpose(c,[2, 0, 1])
            c = c / (cp.sqrt(Rxa) * cp.sqrt(Rxb))
            c = cp.transpose(c, [1,2,0])

            ## SWS estimation post-processing
            # find lag of max correlation
            r = cp.amax(c, axis=2)
            rmax_idx = cp.argmax(c, axis=2)

            # Zero shift
            rmax_idx = cp.abs(rmax_idx - c.shape[2]//2)
            rmax_idx[rmax_idx==0] = 1 

            # Calc SWS
            dt = self.FRI * rmax_idx / self.interp_factor
            sws_map = self.ds / dt

            # Limit the values
            sws_map = cp.clip(sws_map, self.sws_range[0], self.sws_range[1]) 
            
            # Aggregate the results
            print(sws_map.shape)
            print(r.shape)
            SWV[0, i, self.z_clip[0]:-self.z_clip[1], int(cp.ceil(self.xL[i] + self.d/2)) : int(cp.ceil(self.xR[i]-self.d/2))] = sws_map
            SWV[1, i, self.z_clip[0]:-self.z_clip[1], int(cp.ceil(self.xL[i] + self.d/2)) : int(cp.ceil(self.xR[i]-self.d/2))] = r
            # Output format: 4D data: 2x 3D array: SWS: 3D array [dataset, z, x], SWS_r: 3D array [dataset, z, x]. Arrays concat'd along axis=0
            # For example: [SWV/r, LR/RL, z, x]

        return SWV 

In [ ]:
# SWS estimation
#ddata_LR = ddata_LR[10:-30, :, swse_frames[0]:swse_frames[1]] 
#ddata_RL = ddata_RL[10:-30, :, swse_frames[0]:swse_frames[1]] 

In [ ]:
# Overwrite parameters
swse_interp_factor = 10;
swse_interp_order  = 3
swse_d             = 14;
swse_frames        = [0, 99];
swse_SWV_range     = [0.5, 4.0];
swse_x_range       = [[0, 420], [0, 420]]
swse_z_clip        = [10, 30]

In [ ]:
dim = ddata_LR.shape
print(dim)
SWS_Estimator = SWS_Estimation(data_shape=ddata_LR.shape, x_range=swse_x_range, z_clip = swse_z_clip, frames_range = swse_frames,
                               d=swse_d, interp_factor=swse_interp_factor, interp_order=swse_interp_order, 
                                px_pitch=px_size*1e-3, FRI =pwi_fri, sws_range=swse_SWV_range)
SWS_Estimator.prepare()


In [ ]:
# Pack data
ddata_LRi = ddata_LR[np.newaxis, ...]
ddata_RLi = ddata_RL[np.newaxis, ...]
ddata_f  = np.concatenate((ddata_LRi, ddata_RLi), axis=0)
print(ddata_f.shape)
del ddata_LRi
del ddata_RLi

In [ ]:
#%%timeit -r 1 -n 10
SWV = SWS_Estimator.process(data=ddata_f)

In [ ]:
! nvidia-smi

In [ ]:
#Visualize the results
SWV_cpu = SWV.get()
SWS_cpu = np.squeeze(SWV_cpu[0])
SWS_r_cpu = np.squeeze(SWV_cpu[1])
#norm_sws = plt.Normalize(swse_SWV_range[0], swse_SWV_range[1], True)
norm_r   = plt.Normalize(0, 1, True)

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
axs[0,0].imshow(np.squeeze(SWS_cpu[0,:,:]), cmap='jet')#, norm=norm_sws)
axs[0,0].set_title('SWS_LR [m/s]')

axs[1,0].imshow(np.squeeze(SWS_r_cpu[0,:,:]), cmap='jet', norm=norm_r)
axs[1,0].set_title('r_LR')

axs[0,1].imshow(np.squeeze(SWS_cpu[1,:,:]), cmap='jet')#, norm=norm_sws)
axs[0,1].set_title('SWS_RL [m/s]')

axs[1,1].imshow(np.squeeze(SWS_r_cpu[1,:,:]), cmap='jet', norm=norm_r)
axs[1,1].set_title('r_RL')


In [ ]:
# SWS Estimation test
swse_interp_factor = [1, 2, 5, 10, 20]
SWS = []
SWSr = []
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()

SWS_Estimator1 = SWS_Estimation(data_shape=ddata_RL.shape, x_range=swse_x_range[1], d=swse_d, interp_factor=1, interp_order=swse_interp_order, 
                                    px_pitch=px_size*1e-3, FRI =pwi_fri, sws_range=swse_SWV_range)

k=1
for i in swse_interp_factor:
    SWS_Estimator1.interp_factor = i
    SWS_Estimator1.prepare()
    (SWS_data, SWSr_data) = SWS_Estimator1.process(data=ddata_RL)
    SWS.append(SWS_data.get())
    SWSr.append(SWSr_data.get())
    del SWS_data
    del SWSr_data
    SWS_Estimator1.clearCache()
    print(np.ceil(mempool.used_bytes()/ 1024 / 1024))            
    print(np.ceil(mempool.total_bytes()/ 1024 / 1024))  
    print(pinned_mempool.n_free_blocks())    # 1
    mempool.free_all_blocks()
    pinned_mempool.free_all_blocks()
    print(k)
    k = k+1
    


In [ ]:
mempool = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()
print(ddata_RL.nbytes / 1024 / 1024)                   # 400
print(mempool.used_bytes()/ 1024 / 1024)              # 512
print(mempool.total_bytes()/ 1024 / 1024)             # 512
print(pinned_mempool.n_free_blocks())    # 1

mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.used_bytes())              # 0
print(mempool.total_bytes())             # 0
print(pinned_mempool.n_free_blocks())    # 0


In [ ]:
del SWS
del SWSr
cp._default_memory_pool.free_all_blocks()

In [ ]:
! nvidia-smi

# Post-processing

## Image compounding

In [ ]:
print(SWS.shape)
# Pack data
SWS = SWS[np.newaxis, ...]
SWS_r = SWS_r[np.newaxis, ...]

SWS_data = np.concatenate((SWS, SWS_r), axis=0)
print(SWS_data.shape)

In [ ]:
SWS_Compounder = SWS_Compounding()
SWS_Compounder.prepare()

In [ ]:
#%%timeit -r 10 -n 10
(SWSc, SWScr) = SWS_Compounder.process(SWV)

In [ ]:
# Visualize results
SWSc_cpu  = SWSc.get()
SWScr_cpu = SWScr.get()
norm_r   = plt.Normalize(0, 2, True)
norm_sws = plt.Normalize(swse_SWV_range[0], swse_SWV_range[1], True)
norm_sws = plt.Normalize(0, 5, True)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8, 6))
axs[0].imshow(np.squeeze(SWSc_cpu), cmap='jet', norm=norm_sws)
axs[0].set_title('SWSc [m/s]')

axs[1].imshow(np.squeeze(SWScr_cpu), cmap='jet', norm=norm_r)
axs[1].set_title('r [m/s]')

## Image filtering

In [ ]:
MedianFilter = MedianFiltering(kernel_size=median_filter_size)
MedianFilter.prepare()

In [ ]:
#%%timeit -r 10 -n 10
SWS_filtered = MedianFilter.process(SWSc)

## Data presentation

In [ ]:
%%timeit -r 10 -n 10
SWS_filtered_cpu = SWS_filtered.get()

In [ ]:
# Visualize SWS map
plotSWSmap(data=SWS_filtered_cpu[:420, swse_d//2:-swse_d//2], px_size=px_size, tick_grid_size=[5, 5], sws_disp_range=[0, 5])

In [ ]:
A = np.zeros([5,5])
print(A)
print(A.shape)
B = [2, A.shape[0], A.shape[1]]
print(B)
C = np.zeros(B)

In [ ]:
A = np.arange(1, 20)
print(A)
B = A[:-1]
print(B)